In [1]:
# The purpose of this notebook is to batch a large JKG.json for neo4j ingest
# Best to empty the JKG_Batch Directory before running this

In [2]:
# Import python3 modules
import os
import json as json
import pandas as pd
import ijson

In [3]:
# Stream JSON KG from local directory JKG.json and write out batches of up to 1M nodes, rels or CODE-rels into JKG_Batch directory

with open('JKG.json', 'rb') as f:
    i = 0
    for item in ijson.items(f, 'nodes.item'):
        if ((i % 1000000) == 0):
            file = open('JKG_Batch/n'+str(int(i / 1000000)).zfill(4)+'_JKG.json', "w")
            file.write('{"nodes":[\n')
        else:
            file.write('\n,')
        file.write(json.dumps(item))
        if ((i % 1000000) == 999999):
            file.write('\n],"rels":[]}')
            file.close()
        i = i + 1
    if ((i % 1000000) != 0):
        file.write('\n],"rels":[]}')
        file.close()

with open('JKG.json', 'rb') as f:
    i = 0
    firstR = False
    firstCR = False
    for item in ijson.items(f, 'rels.item'):
        if ((i % 1000000) == 0):
            rfile = open('JKG_Batch/r'+str(int(i / 1000000)).zfill(4)+'_JKG.json', "w")
            rfile.write('{"nodes":[],"rels":[\n')
            crfile = open('JKG_Batch/cr'+str(int(i / 1000000)).zfill(4)+'_JKG.json', "w")
            crfile.write('{"nodes":[],"rels":[\n')
            firstCR = True
            firstR = True
        if item['label'] == 'CODE':
            if firstCR == False:
                crfile.write('\n,')
            crfile.write(json.dumps(item))
            firstCR = False
        else:
            if firstR == False:
                rfile.write('\n,')
            rfile.write(json.dumps(item))
            firstR = False
        if ((i % 1000000) == 999999):
            rfile.write('\n]}')
            rfile.close()
            if os.path.getsize(rfile.name) == 24:
                os.remove(rfile.name)
            crfile.write('\n]}')
            crfile.close()
            if os.path.getsize(crfile.name) == 24:
                os.remove(crfile.name)
        i = i + 1
    if ((i % 1000000) != 0):
        rfile.write('\n]}')
        rfile.close()
        if os.path.getsize(rfile.name) == 24:
            os.remove(rfile.name)
        crfile.write('\n]}')
        crfile.close()
        if os.path.getsize(crfile.name) == 24:
            os.remove(crfile.name)